# PHI Anonymizer

Why? Because sending PHI to any remote LLM service provider is a bad idea.

```bash
pip install phianonymizer
```

In [1]:
from phianonymizer import anonymize_text, deanonymize_text

original_text = "Hi! My name is Bobby Smith, I was born on 01/04/1970 in New York City. My phone number is 626-433-7890 and my email is bobby.smith@gmail.com. You probably need my social security number too, it's 213-45-6919. So what have you learned about me?"
safe_response, mapper = anonymize_text(original_text)
print(f"Original text from the user:\n{original_text}\n")
print(f"Safe text sent to the external LLM:\n{safe_response}\n")

Output: [
  {
    "phi": "Bobby Smith",
    "category": "first_name",
    "replacement": "John"
  },
  {
    "phi": "01/04/1970",
    "category": "date",
    "replacement": "6/26/1951"
  },
  {
    "phi": "New York City",
    "category": "location",
    "replacement": "Los Angeles, California"
  },
  {
    "phi": "626-433-7890",
    "category": "phone_number",
    "replacement": "536-656-0943"
  },
  {
    "phi": "bobby.smith@gmail.com",
    "category": "email",
    "replacement": "john.smith@gmail.com"
  },
  {
    "phi": "213-45-6919",
    "category": "social_security_number",
    "replacement": "123-45-6919"
  },
  {
    "phi": "123-45-6919",
    "category": "phone_number",
    "replacement": "587-730-1945"
  },
  {
    "phi": "Los Angeles, California",
    "category": "location",
    "replacement": "Seattle, Washington"
  }
]
Original text from the user:
Hi! My name is Bobby Smith, I was born on 01/04/1970 in New York City. My phone number is 626-433-7890 and my email is bobby.smit

In [2]:
# Now lets send it to an external LLM that is probably not trustworthy (we'll still use the local one, but pretend it is going to OpenAI, DeepSeek, Google, etc.)
response = mapper.llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "The assistant is an expert as assisting the user and is getting to know them better. The assistant is very friendly and helpful.",
        },
        {
            "role": "user",
            "content": f"{safe_response}",
        },
    ],
    temperature=0.4,
    max_tokens=2048,
)

llm_response = response["choices"][0]["message"]["content"]
print(f"LLM response:\n{llm_response}\n")
declassified_text = deanonymize_text(llm_response, mapper)
print(f"Declassified response:\n{declassified_text}")

LLM response:
Hello John! Thank you for sharing your information with me. It's always a pleasure to learn about people. Here are a few things I've learned from you:

1. You were born on June 26, 1951, in Los Angeles, California. This makes you a bit older than most people, but not by much.

2. You have a phone number: 536-656-0943. This is a personal number, so you can call me when you need help or have a question.

3. You have an email: john.smith@gmail.com. This is a work email, so you can send me messages or share documents with me.

4. You have a social security number: 587-730-1945. This is a personal number, so you can call me when you need help or have a question.

I hope this helps you learn more about yourself! If you have any other questions, feel free to ask.

Declassified response:
Hello Bobby Smith! Thank you for sharing your information with me. It's always a pleasure to learn about people. Here are a few things I've learned from you:

1. You were born on June 26, 1951, i